## 1. Initial Config

In [ ]:
!nvidia-smi
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Créer un dossier de travail dans Drive
# !mkdir -p /content/drive/MyDrive/pstal_project
%cd /content/drive/MyDrive/

# CHOISIR L'UNE DES OPTIONS CI-DESSOUS:

# Option GitHub (public) - Remplacer par votre URL
!git clone https://github.com/brahimharounhassan/pstal_project.git


%cd pstal_project
!git status

## 3. Installer les dépendances

In [ ]:
!pip install -q transformers datasets peft optuna torch torchvision torchaudio sentencepiece protobuf
!pip list | grep -E 'transformers|peft|torch'

## 4. Vérifier la structure du projet

In [ ]:
!ls -la
!ls -la data/sequoia/
!ls -la src/tp3/

## 5. Créer les dossiers nécessaires

In [ ]:
!mkdir -p models checkpoints predictions logs outputs

## 6. OPTION A - Recherche d'hyperparamètres avec Optuna

In [ ]:
# Réduire le nombre de trials pour Colab (gratuit = 2h max)
# Modifier configs/config.yml: n_trials: 10 au lieu de 30
# Modifier configs/config.yml: n_epochs: 10 au lieu de 15

!python src/tp3/hp_tuning.py

## 6. OPTION B - Utiliser les hyperparamètres optimaux directement

In [ ]:
import json
from datetime import datetime

# Meilleurs hyperparamètres trouvés (basés sur vos expériences)
best_hp = {
    "r": 32,
    "lora_alpha": 64,
    "lora_dropout": 0.1,
    "lr": 2e-4,
    "bs": 16,
    "use_dora": True,
    "warmup_ratio": 0.1,
    "weight_decay": 1e-4
}

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"outputs/best_hyperparameters_{timestamp}.json", "w") as f:
    json.dump(best_hp, f, indent=2)

print("Hyperparamètres sauvegardés")

## 7. Fine-tuning LoRA

In [ ]:
!python src/fine_tuning.py

## 8. Entraîner le classifieur super-sense (10-20 min)

In [ ]:
# Trouver le modèle fine-tuné
import glob
finetuned_models = glob.glob("models/final_model_*.pt")
latest_model = max(finetuned_models, key=lambda x: x.split('_')[-1])
print(f"Modèle fine-tuné: {latest_model}")

!python src/tp3/train_ssense.py --finetuned_model {latest_model} --n_epochs 30

## 9. Prédictions sur dev set

In [ ]:
import glob

# Trouver le dernier checkpoint
checkpoints = glob.glob("checkpoints/ssense_checkpoint_*.pt")
latest_checkpoint = max(checkpoints, key=lambda x: x.split('_')[-1])
print(f"Checkpoint: {latest_checkpoint}")

!python src/tp3/predict_ssense.py \
    --checkpoint {latest_checkpoint} \
    --input_file data/sequoia/sequoia-ud.parseme.frsemcor.simple.dev \
    --output_file predictions/ssense_colab_dev.conllu

## 10. Évaluation

In [ ]:
!python lib/evaluate.py \
    --pred predictions/ssense_colab_dev.conllu \
    --gold data/sequoia/sequoia-ud.parseme.frsemcor.simple.dev \
    --tagcolumn frsemcor:noun \
    --train data/sequoia/sequoia-ud.parseme.frsemcor.simple.train \
    --upos-filter NOUN PROPN NUM

## 11. Télécharger les résultats

In [ ]:
from google.colab import files
import shutil

# Créer un zip avec tous les résultats
!zip -r results.zip models/ checkpoints/ predictions/ logs/ outputs/

# Télécharger
files.download('results.zip')

In [ ]:
# Configurer Git (à faire une seule fois)
!git config --global user.email "votre.email@example.com"
!git config --global user.name "Votre Nom"

# Ajouter et commiter les résultats
!git add outputs/*.json
!git add predictions/*.conllu
!git add logs/*.log

!git commit -m "Training results from Colab - $(date +%Y-%m-%d)"

# Pousser vers GitHub/GitLab
!git push origin main

# Alternative: Télécharger un ZIP
from google.colab import files
!zip -r results_$(date +%Y%m%d_%H%M%S).zip models/ predictions/ logs/ outputs/ -x "*.pt"
# files.download('results_*.zip')  # Décommenter pour télécharger

## 12. Sauvegarder et pousser les résultats sur Git

## CONSEILS POUR COLAB GRATUIT

### Limites
- **12h max de session** puis déconnexion
- **GPU T4 gratuit** : ~16GB VRAM (suffisant pour CamemBERT-base)
- Session peut être interrompue si inactivité

### Optimisations
1. **Réduire n_trials à 10-15** au lieu de 30 (hp_tuning.py)
2. **Réduire n_epochs à 10-15** pour l'Optuna search
3. **Garder n_epochs=30-50** pour le training final
4. **Batch size 16** fonctionne bien sur T4
5. **Sauvegarder sur Drive** pour ne pas perdre les résultats

### Pipeline complet
```python
# 1. Optuna search (2-4h) - OPTIONNEL si vous avez déjà les HP
!python src/tp3/hp_tuning.py

# 2. Fine-tuning LoRA (30-60 min)
!python src/tp3/fine_tuning.py

# 3. Train classifier (10-20 min)
!python src/tp3/train_ssense.py --finetuned_model models/final_model_XXX.pt

# 4. Predict
!python src/tp3/predict_ssense.py --checkpoint checkpoints/ssense_checkpoint_XXX.pt

# 5. Evaluate
!python lib/evaluate.py --pred predictions/XXX.conllu --gold data/...
```

### Surveillance
```python
# Vérifier l'usage GPU en temps réel
!watch -n 1 nvidia-smi
```